In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
# open text file and read in data as `text`
with open('dostoevsky.txt', 'r') as f:
 text = f.read()
 print(text[:100])

Федор Михайлович Достоевский
Бедные люди
Ох уж эти мне сказочники! Нет чтобы написать что-нибудь пол


токенизация (кодирование символов числами)

In [4]:
# encode the text and map each character to an integer and vice versa
# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
# encode the text
encoded = np.array([char2int[ch] for ch in text])
print(encoded[:100])

[104 112 100  89  32  43  83  74 114  26  12  14  89   8  74  16  43 115
  89  51 120  89 112   8  51  22  74  12  54 103 112 100  11 128 112  43
  14 116 100  74  54  39 114  43  69  82  43  62 120  74  43  79  11 112
  43  51  22  26  88  89  16  11  74  22  74  98  43  45 112 120  43  16
 120  89  59 128  43  11  26  23  74  51  26 120  97  43  16 120  89  68
  11  74  59  69 100  97  43  23  89  14]


Предобработка данных (one-hot кодирование)

In [5]:
def one_hot_encode(arr, n_labels):

 # Initialize the the encoded array
 one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

 # Fill the appropriate elements with ones
 one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

 # Finally reshape it to get back to the original array
 one_hot = one_hot.reshape((*arr.shape, n_labels))

 return one_hot

In [6]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)
print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [7]:
def get_batches(arr, batch_size, seq_length):
 '''Create a generator that returns batches of size
 batch_size x seq_length from arr.

 Arguments
 ---------
 arr: Array you want to make batches from
 batch_size: Batch size, the number of sequences per batch
 seq_length: Number of encoded chars in a sequence
 '''

 ## TODO: Get the number of batches we can make
 n_batches =len(arr)//(batch_size*seq_length)

 ## TODO: Keep only enough characters to make full batches
 arr =arr[:n_batches*batch_size*seq_length]

 ## TODO: Reshape into batch_size rows
 arr =arr.reshape([batch_size, -1])

 ## Iterate over the batches using a window of size seq_length
 for n in range(0, arr.shape[1], seq_length):
 # The features
  x = arr[:, n:n+seq_length]
 # The targets, shifted by one
  y = np.zeros_like(x)
  try:
    y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
  except IndexError:
    y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
  yield x, y


In [8]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])


x
 [[104 112 100  89  32  43  83  74 114  26]
 [ 14  40  43  69  14 128  59  26  29  51]
 [ 43   8  43  23  32  74   8 128  16  11]
 [ 51  97  43  89  51 120  26   8  26 120]
 [100  97  43  11  26  79 112  32 112  11]
 [ 43  51 112  67  89 100  11  29  43 100]
 [ 11 112  43  51  89   8  51 112  79  43]
 [ 74  43  23  89  23  26  14  89  40  43]]

y
 [[112 100  89  32  43  83  74 114  26  12]
 [ 40  43  69  14 128  59  26  29  51  97]
 [  8  43  23  32  74   8 128  16  11  89]
 [ 97  43  89  51 120  26   8  26 120  97]
 [ 97  43  11  26  79 112  32 112  11  74]
 [ 51 112  67  89 100  11  29  43 100  89]
 [112  43  51  89   8  51 112  79  43 112]
 [ 43  23  89  23  26  14  89  40  43  26]]


In [9]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
  print('Training on GPU!')
else:
  print('No GPU available, training on CPU; consider making n_epochs very small.')
class CharRNN(nn.Module):
  def __init__(self, tokens, n_hidden=256, n_layers=3, drop_prob=0.5, lr=0.001):
    super().__init__()
    self.drop_prob = drop_prob
    self.n_layers = n_layers
    self.n_hidden = n_hidden
    self.lr = lr

    # creating character dictionaries
    self.chars = tokens
    self.int2char = dict(enumerate(self.chars))
    self.char2int = {ch: ii for ii, ch in self.int2char.items()}

    ## TODO: define the layers of the model
    self.dropout = nn.Dropout(drop_prob);

    self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True)

    self.fc =nn.Linear(n_hidden, len(self.chars))


  def forward(self, x, hidden):
    # TODO: Get the outputs and the new hidden state from the lstm
    r_output, hidden = self.lstm(x, hidden)

    ## TODO: pass through a dropout layer
    out =self.dropout(r_output)

    #Stack up LSTM output using view
    #you may need to use contiguous to reshape the output
    out =out.contiguous().view(-1, self.n_hidden)

    ## TODO: put x through the fully-connected layer
    out = self.fc(out)

    # return the final output and the hidden state
    return out, hidden
  def init_hidden(self, batch_size):
    ''' Initializes hidden state '''
    # Create two new tensors with sizes n_layers x batch_size x n_hidden,
    # initialized to zero, for hidden state and cell state of LSTM
    weight = next(self.parameters()).data

    if (train_on_gpu):
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
    else:
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

    return hidden




Training on GPU!


Тренировка

In [10]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))

                    val_losses.append(val_loss.item())

                net.train() # reset to train mode after iterationg through validation data

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

Обучение

In [11]:
# define and print the net
n_hidden=512
n_layers=3

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(133, 512, num_layers=3, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=133, bias=True)
)


In [12]:
batch_size = 64
seq_length = 100 #max length verses
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.3643... Val Loss: 3.3240
Epoch: 1/50... Step: 20... Loss: 3.3259... Val Loss: 3.2938
Epoch: 1/50... Step: 30... Loss: 3.2964... Val Loss: 3.2849
Epoch: 1/50... Step: 40... Loss: 3.2586... Val Loss: 3.2850
Epoch: 1/50... Step: 50... Loss: 3.2754... Val Loss: 3.2793
Epoch: 1/50... Step: 60... Loss: 3.2276... Val Loss: 3.2816
Epoch: 2/50... Step: 70... Loss: 3.2366... Val Loss: 3.2840
Epoch: 2/50... Step: 80... Loss: 3.2335... Val Loss: 3.2813
Epoch: 2/50... Step: 90... Loss: 3.2276... Val Loss: 3.2849
Epoch: 2/50... Step: 100... Loss: 3.2349... Val Loss: 3.2850
Epoch: 2/50... Step: 110... Loss: 3.2284... Val Loss: 3.2838
Epoch: 2/50... Step: 120... Loss: 3.2378... Val Loss: 3.2807
Epoch: 3/50... Step: 130... Loss: 3.2231... Val Loss: 3.2824
Epoch: 3/50... Step: 140... Loss: 3.2148... Val Loss: 3.2729
Epoch: 3/50... Step: 150... Loss: 3.2205... Val Loss: 3.2457
Epoch: 3/50... Step: 160... Loss: 3.1236... Val Loss: 3.2049
Epoch: 3/50... Step: 170... Loss:

Сохранение модели

In [13]:
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

Предсказание. Сэмплирование.

In [14]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''

        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)

        if(train_on_gpu):
            inputs = inputs.cuda()

        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # get top characters
        # considering the k most probable characters with topk method
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [15]:
def sample(net, size, prime='The', top_k=None):

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()

    net.eval() # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [16]:
print(sample(net, 1000, prime='Анна', top_k=5))

Анная, что вы сами не забыли его!.. Об этом не смотрите, что я не смею совсем свое рассказать.
Ваш их добрый, никогда ни старико не смел подомнил и, все такая женщена. Письмо переменятся, право, потому что он не могу ничего. Вставал вы мои совершенно в то же никогда на вестет, и перепомнились, как я все это будет, теперь подошно покупаете, а все так и трудно будет? Я не стала после, по крайней мере переменить свою сердце, а все это бы со мной не знали, все сказать, когда я сам совестно. Вы меня ни наконец он посыл и с ним под кухла на всякой случае. Наконец она не понимает, что-то сердце и все только потом. Я так же случалась, так что вот вот, не стел не спать, и в таком не знаете, что она проходили положинить, чтоб я не пришел под пред нами старика, не слыша было с совершенною наша иструтить свое видным душевной. Я последнюе весьма решенок вы свои дело на все письма, так ведь я подумал. Я начал меня в нет такой простой, подумайте, что он не продолжали свою руку и делал с ним и должнос

Загрузка модели

In [18]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="И он сказал"))

<ipython-input-18-ad4007b597b5>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f)


И он сказал; что он вышел в том! что он приходил меня по себя не могла, не предудательные, что у меня пред прочли, как же все стоять под досадного, что все это новые, как он в таком старенном совственнему, что вы не мог следы и темною просто время. Я ведь сам не смел на свои сердце, что вы напечатате? 
— Ну, не сказал! 
От манушки, нужно быть такая непременно письмо не могу станете! Нет, ведь только я не знаю, что я сам стоял с капмам. Я весь представилось ни в половеную себе свой привыкаешься, тужаться и собрались. Прислали прохожий не смел не забыла. Я вам ведь не подновила свои приступительно в комнату, и нет ни дума, то я все выражение не смысли, что он сердется. Но подумайте, чем мое слово бы не могу. Вы только посылаю в свою ветреное, то, что он не знаю, что я сама сердце представил предумательного ни с полениной придум на слова и сказал меня. Ну, что он только что становится на все время, как бы и всякий про себя, то все такие надостите меня с пред плочем ничего. Обедались; вы т